In [2]:
import math
from functools import reduce
from IPython.display import display, Markdown

# text = "|hi|234|\n|-|-|\n|-|asdf|"
# display(Markdown(text))
def printmd(text):
  display(Markdown(text))

##Ввод данных

In [1]:
arr = [
    "10",
    "3",
    "2 7", #кукуруза
    "0 100",
    "2 10", #помидоры
    "0 2",
    "3 3", #бобы
    "1 100",
]

In [21]:
arr = [
    "10",
    "3",
    "2 14",
    "0 2",
    "4 40",
    "0 100",
    "8 31",
    "0 1",
]

In [18]:
arr = [
    "4",
    "3",
    "1 14",
    "0 100",
    "3 47",
    "0 100",
    "2 31",
    "0 200",
]

In [34]:
arr = [
    "0.5",
    "3",
    "0.1 14",
    "1 1",
    "0.3 40",
    "1 100",
    "0.2 31",
    "0 100",
]

In [15]:
read_index = -1
def read(msg):
  print(msg)
  global read_index
  read_index+=1
  return arr[read_index]

In [ ]:
# def read(msg):
  # return input(msg)

In [35]:
read_index = -1

real_weight = float(read("Total weight")) #вес всего рюкзака
weight = real_weight
num_items = int(read("Number of items")) #количество предметов
items = []
limits = [] # нижняя и верхняя граница по количеству предметов
all_weightes = [] #для подсчёта шага
for i in range(num_items):
  # вес и цена каждого предмета
  wi, pi = map(lambda x: float(x), read("Wi and Pi").split())
  l_lim, r_lim = map(lambda x: int(x), read("min and max Ui (limits)").split()) #правое и левое ограничение
  all_weightes.append(wi)
  items.append([wi,pi])
  limits.append([l_lim, r_lim-l_lim])
  weight -= wi*l_lim
weight = round(weight,4)
all_weightes.append(weight)
print(all_weightes)
print(items)
print(limits)

Total weight
Number of items
Wi and Pi
min and max Ui (limits)
Wi and Pi
min and max Ui (limits)
Wi and Pi
min and max Ui (limits)
[0.1, 0.3, 0.2, 0.1]
[[0.1, 14.0], [0.3, 40.0], [0.2, 31.0]]
[[1, 0], [1, 99], [0, 100]]


##Подготовка

In [39]:
def nod(a,b):
  if a == 0:
    return b
  elif b == 0:
    return a
  elif a>b:
    return nod(a%b,b)
  else:
    return nod(b%a,a)

max_digits = 0
for i in range(len(all_weightes)):
  digits = 0
  s = str(all_weightes[i])
  if "." in s:
    digits = len(s.split(".")[1])
  if digits > max_digits:
    max_digits = digits


ratio = 10 ** max_digits #на сколько все числа домножаем
step = reduce(nod, map(lambda x: int(x*ratio), all_weightes))/ratio
print(step)

answers = [[]] #массивы с f(x)* и u* для каждого этапа
iterations = math.ceil(weight/step)+1 #количество строк в каждой таблице. +1 т.к. с нулём

for i in range(iterations):
  answers[0].append([0]) #заполяняем первый массив нулями, чтобы первая итерация работала (костыль)

0.1


##Решение

In [40]:
temp = []
text = ""
for item_index in range(num_items):
  temp = []
  wi, pi = items[item_index] #вес и цена каждого из предметов
  r_limit = limits[item_index][1] #сколько максимум мы можем положить предметов
  total_u = min(r_limit, int((iterations-1)*step/wi))
  # print(wi, pi, r_limit)
  text += f"\n\n##Stage {item_index+1}\n"
  i = num_items-item_index
  text += f"|i|X{i}"
  for j in range(total_u+1):
    text += f"|U{i}={j}"
  text += f"|F(x)*|U{i}*,Pi|\n"
  text += "|-|-"
  for j in range(total_u+1):
    text += "|-"
  text += "|-|-|\n"
  for i in range(iterations): #проход по строкам таблицы в тетради
    xi = i*step #сколько у нас сейчас доступно веса
    text += f"|{i}|{round(xi,4)}|"
    ui = 1 #количество предметов
    max_f = answers[item_index][i][0] #max_f - наибольшее значение ценности f(x)*
    max_u = [(0, i)] #количество при наибольшем значении ценности u*
    text += f"{max_f}|"
    while (ui*wi<=xi and ui<=r_limit):
      prev_index = int(i-ui*wi/step) # индекс для значения f(x) с предыдушего этапа
      fi = ui*pi + answers[item_index][prev_index][0] #текущее значение функции + с предыдущего этапа
      text += f"{fi}|"
      if max_f < fi:
        max_f = fi
        max_u = [(ui, prev_index)]
      elif max_f == fi:
        max_u.append((ui, prev_index))
      ui+=1
    temp.append([max_f,max_u])
    for j in range(ui,total_u+1):
      text += "-|"
    # if total_u-ui>0 or True:
      # text += "|"
    text += f"{max_f}|{', '.join(map(str,max_u))}|\n"

  answers.append(temp)

printmd(text) 



##Stage 1
|i|X3|U3=0|F(x)*|U3*,Pi|
|-|-|-|-|-|
|0|0.0|0|0|(0, 0)|
|1|0.1|0|0|(0, 1)|


##Stage 2
|i|X2|U2=0|F(x)*|U2*,Pi|
|-|-|-|-|-|
|0|0.0|0|0|(0, 0)|
|1|0.1|0|0|(0, 1)|


##Stage 3
|i|X1|U1=0|F(x)*|U1*,Pi|
|-|-|-|-|-|
|0|0.0|0|0|(0, 0)|
|1|0.1|0|0|(0, 1)|


##Ответ

In [41]:
#функция, которая возвращает массив с ответами (сколько каждого предмета нужно взять)
def count_items(x_index, item_index):
  result = []
  ways = answers[item_index][x_index][1]
  if item_index == 1:
    for way in ways:
      result.append([way[0]])
  else:
    for way in ways:
      ui, next_index = way
      temp = count_items(next_index,item_index-1)
      for x in temp:
        x.append(ui)
      result += temp
  return result

#вывод ответа
text = ""
max_index = -1
for i,x in enumerate(answers[-1]):
  if x[0] == max_f:
    max_index = i
    break
result = count_items(max_index, num_items)
for i in range(num_items):
  l_lim = limits[i][0]
  for x in result:
    x[i]+=l_lim
text += "|№|"
for i in range(num_items):
  text += f"{i}|"
text += "\n|-:|"
for i in range(num_items):
  text += f"-|"
text += "\n|**Weight**|"
for x in items:
  text += f"{x[0]}|"
text += "\n|**Price**|"
for x in items:
  text += f"{x[1]}|"

for i in range(len(result)):
  text += f"\n|**Solution {i+1}**|"
  text += '|'.join(map(str,result[i]))
  text += '|'

final_price = 0
final_weight = 0
for i, ui in enumerate(result[0]):
  wi, pi = items[i]
  final_price += pi*ui
  final_weight += wi*ui

text += '\n\n## Results\n'
text += '|Final price|Final weight|\n'
text += '|:-:|:-:|\n'
text += f"|{final_price}|{final_weight}/{real_weight}|"

printmd(text)

|№|0|1|2|
|-:|-|-|-|
|**Weight**|0.1|0.3|0.2|
|**Price**|14.0|40.0|31.0|
|**Solution 1**|1|1|0|

## Results
|Final price|Final weight|
|:-:|:-:|
|54.0|0.4/0.5|